In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, roc_curve
import shap
from sklearn.inspection import permutation_importance
from sklearn.inspection import PartialDependenceDisplay
# from imblearn.over_sampling import RandomOverSampler
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 200)

d:\JAN\Program Files\Conda\envs\mastercard\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
merchants = pd.read_csv('../data/merchants.csv')
users = pd.read_csv('../data/users.csv')
transactions = pd.read_json('../data/transactions.json', lines=True, dtype_backend="numpy_nullable")

df_combined = transactions.merge(merchants, on='merchant_id', how='left')
df_combined = df_combined.merge(users, on='user_id', how='left')

In [ ]:
bad_rate_by_id = transactions.groupby('merchant_id')['is_fraud'].agg(['count', 'sum'])
bad_rate_by_id['bad_rate'] = bad_rate_by_id['sum'] / bad_rate_by_id['count']

bad_rate_sorted = bad_rate_by_id.sort_values('bad_rate', ascending=False)

print(" Top 5 IDs with highest % of bads:")
print(bad_rate_sorted.head(5))

print("\n Bottom 5 IDs with lowest % of bads:")
print(bad_rate_sorted.tail(5))

🔝 Top 5 IDs with highest % of bads:
             count  sum  bad_rate
merchant_id                      
M0167          570   84  0.147368
M0384          511   75  0.146771
M0916          502   73  0.145418
M0794          534   77  0.144195
M0818          472   68  0.144068

🔻 Bottom 5 IDs with lowest % of bads:
             count  sum  bad_rate
merchant_id                      
M0880          503   20  0.039761
M0699          472   18  0.038136
M0895          510   18  0.035294
M0436          511   17  0.033268
M0110          511   16  0.031311


In [4]:
df_combined['timestamp'] = pd.to_datetime(df_combined['timestamp'])
df_combined['Month_Year_EOM'] = (df_combined['timestamp'] + pd.offsets.MonthEnd(0)).dt.date
df_combined['Date'] = df_combined['timestamp'].dt.strftime('%d-%m-%Y')
df_combined['Year'] = df_combined['timestamp'].dt.strftime('%Y')

def get_part_of_day(hour):
    if 6 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 17:
        return 'afternoon'
    elif 17 <= hour < 21:
        return 'evening'
    elif 21 <= hour < 23:
        return 'night'
    else:
        return 'late_night'

df_combined['hour'] = df_combined['timestamp'].dt.hour
df_combined['part_of_day'] = df_combined['hour'].apply(get_part_of_day)

In [5]:
df = df_combined

In [ ]:
df = df.sort_values(['user_id', 'timestamp'])

df['time_diff'] = df.groupby('user_id')['timestamp'].diff()

In [7]:
df['time_diff_hours'] = (df['time_diff'].dt.total_seconds() / 3600).round(2)
df['latitude'] = df['location'].apply(lambda x: x.get('lat')).round(2)
df['longitude'] = df['location'].apply(lambda x: x.get('long')).round(2)

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of Earth in kilometers. 
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    return R * c  

In [ ]:

df = df.sort_values(['user_id', 'timestamp'])

df['lat_prev'] = df.groupby('user_id')['latitude'].shift()
df['lon_prev'] = df.groupby('user_id')['longitude'].shift()
df['time_prev'] = df.groupby('user_id')['timestamp'].shift()

df['time_diff_hours'] = (df['timestamp'] - df['time_prev']).dt.total_seconds() / 3600

df['distance_km'] = haversine(df['lat_prev'], df['lon_prev'], df['latitude'], df['longitude'])

df['speed_kmph'] = df['distance_km'] / df['time_diff_hours']

In [10]:
df['amount/avg_amount'] = df['amount'] / df['avg_transaction_amount']
df['amount/sum_monthly_installments'] = df['amount'] / df['sum_of_monthly_installments']
df['amount/sum_monthly_expenses'] = df['amount'] / df['sum_of_monthly_expenses']
df['speed_kmph'] = df['speed_kmph'].clip(upper=2000)
df['countries_same'] = (df['country_x'] == df['country_y']).astype(int)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def calculate_bad_percentage_with_plot(df, feature, target):
    """
    Calculates bad rate per category and plots a bar chart.

    Parameters:
    - df: pandas DataFrame
    - feature: categorical column name
    - target: binary column name (1 = bad, 0 = good)

    Returns:
    - summary DataFrame with total, bads, and bad_rate columns
    """
    if feature not in df.columns or target not in df.columns:
        raise ValueError(f"Columns '{feature}' or '{target}' not found in DataFrame.")

    grouped = df.groupby(feature)[target].agg(['count', 'sum']).reset_index()
    grouped.rename(columns={'count': 'total', 'sum': 'bads'}, inplace=True)
    grouped['bad_rate'] = grouped['bads'] / grouped['total']

    if 'bad_rate' not in grouped.columns:
        raise ValueError("'bad_rate' column was not created. Check input data.")

    # Plotting
    plt.figure(figsize=(10, 6))
    plt.bar(grouped[feature].astype(str), grouped['bad_rate'])
    plt.xlabel(feature)
    plt.ylabel('Bad Rate')
    plt.title(f'Bad Rate by {feature}')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    return grouped



In [12]:
df.head()

,transaction_id,timestamp,user_id,merchant_id,amount,channel,currency,device,location,payment_method,is_international,session_length_seconds,is_first_time_merchant,is_fraud,category,country_x,trust_score,number_of_alerts_last_6_months,avg_transaction_amount,account_age_months,has_fraud_history,age,sex,education,primary_source_of_income,sum_of_monthly_installments,sum_of_monthly_expenses,country_y,signup_date,risk_score,Month_Year_EOM,Date,Year,hour,part_of_day,time_diff,time_diff_hours,latitude,longitude,lat_prev,lon_prev,time_prev,distance_km,speed_kmph,amount/avg_amount,amount/sum_monthly_installments,amount/sum_monthly_expenses,countries_same
290822,TX290822,2022-01-08 10:03:00,U00001,M0906,19.09,online,EUR,Web,"{'lat': 69.291741, 'long': -7.72625}",debit_card,1,811,0,0,grocery,Belgium,0.631421,2,26.85,37,0,56,Other,High School,Employment,477.69,243.18,Finland,2021-04-01,0.571079,2022-01-31,08-01-2022,2022,10,morning,NaT,NaN,69.29,-7.73,NaN,NaN,NaT,NaN,NaN,0.710987,0.039963,0.078502,0
197951,TX197951,2022-01-08 16:48:00,U00001,M0245,32.24,online,EUR,Android,"{'lat': 54.710715, 'long': 15.368695}",mobile_payment,1,233,1,0,gaming,Spain,0.859810,2,197.43,115,1,56,Other,High School,Employment,477.69,243.18,Finland,2021-04-01,0.571079,2022-01-31,08-01-2022,2022,16,afternoon,0 days 06:45:00,6.750000,54.71,15.37,69.29,-7.73,2022-01-08 10:03:00,1994.074291,295.418413,0.163298,0.067491,0.132577,0
187979,TX187979,2022-01-12 16:23:00,U00001,M0332,4.29,in-store,EUR,Android,"{'lat': 58.897717, 'long': 30.25543}",debit_card,1,521,1,0,grocery,Spain,0.934815,5,18.36,68,0,56,Other,High School,Employment,477.69,243.18,Finland,2021-04-01,0.571079,2022-01-31,12-01-2022,2022,16,afternoon,3 days 23:35:00,95.583333,58.90,30.26,54.71,15.37,2022-01-08 16:48:00,1016.169374,10.631240,0.23366,0.008981,0.017641,0
137034,TX137034,2022-01-18 10:40:00,U00001,M0331,67.0,in-store,EUR,iOS,"{'lat': 42.684202, 'long': 3.843828}",debit_card,1,1149,0,0,gaming,Belgium,0.859404,6,123.91,94,1,56,Other,High School,Employment,477.69,243.18,Finland,2021-04-01,0.571079,2022-01-31,18-01-2022,2022,10,morning,5 days 18:17:00,138.283333,42.68,3.84,58.90,30.26,2022-01-12 16:23:00,2556.977104,18.490855,0.540715,0.140258,0.275516,0
401318,TX401318,2022-07-18 08:05:00,U00001,M0971,3.59,in-store,EUR,Web,"{'lat': 44.296089, 'long': 15.749937}",mobile_payment,1,261,1,0,clothing,Germany,0.516453,1,0.98,52,1,56,Other,High School,Employment,477.69,243.18,Finland,2021-04-01,0.571079,2022-07-31,18-07-2022,2022,8,morning,180 days 21:25:00,4341.416667,44.30,15.75,42.68,3.84,2022-01-18 10:40:00,976.612531,0.224952,3.663265,0.007515,0.014763,0


In [ ]:


geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
geo_df = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

geo_df = gpd.sjoin(geo_df, world[['geometry', 'name']], how='left', predicate='within')

geo_df = geo_df.rename(columns={'name': 'transaction_country'})

print(geo_df[['latitude', 'longitude', 'transaction_country']])

C:\Users\Janek\AppData\Local\Temp\ipykernel_23836\2445756345.py:10: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
d:\JAN\Program Files\Conda\envs\mastercard\lib\site-packages\pyogrio\core.py:35: RuntimeWarning: Could not detect GDAL data files.  Set GDAL_DATA environment variable to the correct path.
  _init_gdal_data()


        latitude  longitude transaction_country
290822     69.29      -7.73                 NaN
197951     54.71      15.37                 NaN
187979     58.90      30.26              Russia
137034     42.68       3.84                 NaN
401318     44.30      15.75             Croatia
...          ...        ...                 ...
263172     58.08      36.67              Russia
360206     50.80      16.07              Poland
358294     37.93      -3.93               Spain
251399     64.19      -8.16                 NaN
368254     58.77      10.27                 NaN

[500000 rows x 3 columns]


In [15]:
geo_df['is_country_nan'] = geo_df['transaction_country'].isna()
geo_df['country_merchant'] = geo_df['country_x']
geo_df['country_user'] = geo_df['country_y']


In [ ]:
import numpy as np
from geopy.distance import geodesic


world['centroid'] = world.geometry.centroid                    

country_to_centroid = world.set_index('name')['centroid'].to_dict()

def centroid_lat_lon(country):
    pt = country_to_centroid.get(country)
    if pt is None or pd.isna(country):
        return pd.Series({'centroid_lat': np.nan, 'centroid_lon': np.nan})
    return pd.Series({'centroid_lat': pt.y, 'centroid_lon': pt.x})  


geo_df[['merchant_centroid_lat', 'merchant_centroid_lon']] = (
    geo_df['country_merchant'].apply(centroid_lat_lon)
)

geo_df[['user_centroid_lat', 'user_centroid_lon']] = (
    geo_df['country_user'].apply(centroid_lat_lon)
)


def distance_km(lat1, lon1, lat2, lon2):
    if np.isnan(lat2) or np.isnan(lon2):
        return np.nan
    return geodesic((lat1, lon1), (lat2, lon2)).kilometers

geo_df['country_merchant_distance_centroid'] = geo_df.apply(
    lambda r: distance_km(r.latitude, r.longitude,
                          r.merchant_centroid_lat, r.merchant_centroid_lon),
    axis=1
)

geo_df['country_user_distance_centroid'] = geo_df.apply(
    lambda r: distance_km(r.latitude, r.longitude,
                          r.user_centroid_lat, r.user_centroid_lon),
    axis=1
)


print(
    geo_df[[
        'latitude', 'longitude',
        'country_merchant', 'merchant_centroid_lat', 'merchant_centroid_lon',
        'country_merchant_distance_centroid',
        'country_user', 'user_centroid_lat', 'user_centroid_lon',
        'country_user_distance_centroid'
    ]].head()
)


C:\Users\Janek\AppData\Local\Temp\ipykernel_23836\1666530175.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  world['centroid'] = world.geometry.centroid                     # shapely Point (lon, lat)


        latitude  longitude country_merchant  merchant_centroid_lat  \
290822     69.29      -7.73          Belgium              50.652441   
197951     54.71      15.37            Spain              40.348656   
187979     58.90      30.26            Spain              40.348656   
137034     42.68       3.84          Belgium              50.652441   
401318     44.30      15.75          Germany              51.133723   

        merchant_centroid_lon  country_merchant_distance_centroid  \
290822               4.580834                         2177.313231   
197951              -3.617021                         2129.369397   
187979              -3.617021                         3142.670637   
137034               4.580834                          888.049093   
401318              10.288485                          862.694031   

       country_user  user_centroid_lat  user_centroid_lon  \
290822      Finland          64.504094          26.211765   
197951      Finland          64.5040

In [ ]:
import numpy as np
from geopy.distance import geodesic

widths  = []
heights = []

for geom in world.geometry:
    minx, miny, maxx, maxy = geom.bounds
    widths.append( geodesic((miny, minx), (miny, maxx)).kilometers )   
    heights.append( geodesic((miny, minx), (maxy, minx)).kilometers )  

world['country_width_km']  = widths
world['country_height_km'] = heights
world['country_size_km']   = (world['country_width_km'] + world['country_height_km']) / 2

size_lookup = world.set_index('name')['country_size_km']

geo_df['merchant_country_size_km'] = geo_df['country_merchant'].map(size_lookup)
geo_df['user_country_size_km']     = geo_df['country_user'].map(size_lookup)

geo_df['country_merchant_distance_centroid_w'] = (
    geo_df['country_merchant_distance_centroid'] / geo_df['merchant_country_size_km']
)

geo_df['country_user_distance_centroid_w'] = (
    geo_df['country_user_distance_centroid'] / geo_df['user_country_size_km']
)

print(geo_df[['country_merchant_distance_centroid_w',
              'country_user_distance_centroid_w']].head())


        country_merchant_distance_centroid_w  country_user_distance_centroid_w
290822                              9.069727                          1.768340
197951                              2.143134                          1.418038
187979                              3.162986                          0.750300
137034                              3.699221                          3.196029
401318                              1.122667                          2.662865


In [23]:
geo_df[geo_df['transaction_country'].isna()].head()

,transaction_id,timestamp,user_id,merchant_id,amount,channel,currency,device,location,payment_method,is_international,session_length_seconds,is_first_time_merchant,is_fraud,category,country_x,trust_score,number_of_alerts_last_6_months,avg_transaction_amount,account_age_months,has_fraud_history,age,sex,education,primary_source_of_income,sum_of_monthly_installments,sum_of_monthly_expenses,country_y,signup_date,risk_score,Month_Year_EOM,Date,Year,hour,part_of_day,time_diff,time_diff_hours,latitude,longitude,lat_prev,lon_prev,time_prev,distance_km,speed_kmph,amount/avg_amount,amount/sum_monthly_installments,amount/sum_monthly_expenses,countries_same,geometry,index_right,transaction_country,is_country_nan,country_merchant,country_user,merchant_centroid_lat,merchant_centroid_lon,user_centroid_lat,user_centroid_lon,country_merchant_distance_centroid,country_user_distance_centroid,merchant_country_size_km,user_country_size_km,country_merchant_distance_centroid_w,country_user_distance_centroid_w
290822,TX290822,2022-01-08 10:03:00,U00001,M0906,19.09,online,EUR,Web,"{'lat': 69.291741, 'long': -7.72625}",debit_card,1,811,0,0,grocery,Belgium,0.631421,2,26.85,37,0,56,Other,High School,Employment,477.69,243.18,Finland,2021-04-01,0.571079,2022-01-31,08-01-2022,2022,10,morning,NaT,NaN,69.29,-7.73,NaN,NaN,NaT,NaN,NaN,0.710987,0.039963,0.078502,0,POINT (-7.73000 69.29000),NaN,NaN,True,Belgium,Finland,50.652441,4.580834,64.504094,26.211765,2177.313231,1555.291561,240.063811,879.520844,9.069727,1.768340
197951,TX197951,2022-01-08 16:48:00,U00001,M0245,32.24,online,EUR,Android,"{'lat': 54.710715, 'long': 15.368695}",mobile_payment,1,233,1,0,gaming,Spain,0.859810,2,197.43,115,1,56,Other,High School,Employment,477.69,243.18,Finland,2021-04-01,0.571079,2022-01-31,08-01-2022,2022,16,afternoon,0 days 06:45:00,6.750000,54.71,15.37,69.29,-7.73,2022-01-08 10:03:00,1994.074291,295.418413,0.163298,0.067491,0.132577,0,POINT (15.37000 54.71000),NaN,NaN,True,Spain,Finland,40.348656,-3.617021,64.504094,26.211765,2129.369397,1247.193753,993.577138,879.520844,2.143134,1.418038
137034,TX137034,2022-01-18 10:40:00,U00001,M0331,67.0,in-store,EUR,iOS,"{'lat': 42.684202, 'long': 3.843828}",debit_card,1,1149,0,0,gaming,Belgium,0.859404,6,123.91,94,1,56,Other,High School,Employment,477.69,243.18,Finland,2021-04-01,0.571079,2022-01-31,18-01-2022,2022,10,morning,5 days 18:17:00,138.283333,42.68,3.84,58.90,30.26,2022-01-12 16:23:00,2556.977104,18.490855,0.540715,0.140258,0.275516,0,POINT (3.84000 42.68000),NaN,NaN,True,Belgium,Finland,50.652441,4.580834,64.504094,26.211765,888.049093,2810.974138,240.063811,879.520844,3.699221,3.196029
432505,TX432505,2022-11-14 19:10:00,U00001,M0209,17.39,in-store,EUR,iOS,"{'lat': 67.012334, 'long': 9.76158}",debit_card,1,591,0,0,grocery,Belgium,0.751451,3,74.74,119,0,56,Other,High School,Employment,477.69,243.18,Finland,2021-04-01,0.571079,2022-11-30,14-11-2022,2022,19,evening,86 days 00:11:00,2064.183333,67.01,9.76,63.06,29.89,2022-08-20 18:59:00,1035.702485,0.501749,0.232673,0.036404,0.071511,0,POINT (9.76000 67.01000),NaN,NaN,True,Belgium,Finland,50.652441,4.580834,64.504094,26.211765,1844.900982,801.280362,240.063811,879.520844,7.685044,0.911042
381637,TX381637,2022-12-10 09:06:00,U00001,M0441,12.95,mobile,EUR,Web,"{'lat': 45.338156, 'long': 36.774688}",credit_card,1,742,1,1,education,Germany,0.796304,3,8.99,92,1,56,Other,High School,Employment,477.69,243.18,Finland,2021-04-01,0.571079,2022-12-31,10-12-2022,2022,9,morning,25 days 13:56:00,613.933333,45.34,36.77,67.01,9.76,2022-11-14 19:10:00,2882.508980,4.695150,1.440489,0.02711,0.053253,0,POINT (36.77000 45.34000),NaN,NaN,True,Germany,Finland,51.133723,10.288485,64.504094,26.211765,2056.795880,2231.132924,768.433024,879.520844,2.676610,2.536760


In [32]:
columns_to_save = ['transaction_id', 'transaction_country',
                   'is_country_nan', 'country_merchant_distance_centroid',
                   'country_user_distance_centroid', 'country_merchant_distance_centroid_w', 
                   'country_user_distance_centroid_w']
df_to_save = geo_df[columns_to_save]

In [33]:
import os

os.makedirs("../data", exist_ok=True)

df_to_save.to_csv("../data/geo_df.csv", index=False)
